# Notebook to compare/analyse mask

### ACM (Auto Clear Mask): cloud mask generated by EarthDailyAgro
### WQR (World Quality Reference): Manual mask based on ACM mask correction by an operator 
### SCL (Sentinel-2 L2A Scene Classification)

In [ ]:
from rasterio.session import AWSSession
import boto3
import pandas as pd
import numpy as np
from utils import list_available_data, compute_accuracy_from_df, compute_accuracy_from_path, download_s3_files

AWS_ACCESS_KEY_ID = '*****'
AWS_SECRET_ACCESS_KEY = '*****'

boto3_session = boto3.session.Session(
    region_name='us-east-2',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

aws_session = AWSSession(requester_pays=True)

#### list available data on a dedicated bucket in AWS S3

In [ ]:
s3_client = boto3.client('s3')
# bucket test
bucket_name = 'clear-mask-quality-comparer'

df = list_available_data(bucket_name)
df

# compute matrix confusion

## mask SCL
#### Label 0 : no_data
#### Label 1,2,4,5,6,7 : clear
#### Label 3,8,9,10,11 : cloud


In [ ]:
acm_vs_wqr, scl_vs_wqr, scl_vs_acm = compute_accuracy_from_df(aws_session,df)

#### compute confusion matrix on a specific dataset

In [ ]:
im = 3

acm_path = df['ACM'][im] # 's3://clear-mask-quality-comparer/S2B_T31TCJ_20230219_105009_UTM31/S2B_T31TCJ_20230219_105009_utm31_gs2_acm.tif'
scl_path = df['SCL'][im] # 's3://sentinel-s2-l2a/tiles/31/T/CJ/2023/2/19/0/R20m/SCL.jp2'
wqr_path = df['WQR'][im] # 's3://clear-mask-quality-comparer/S2B_T31TCJ_20230219_105009_UTM31/S2B_T31TCJ_20230219_105009_utm31_gs2_mask.tif'

acm_vs_wqr, scl_vs_wqr, scl_vs_acm = compute_accuracy_from_path(aws_session, acm_path, scl_path, wqr_path)

#### download data from AWS

In [ ]:
# to download the reference mask
download_s3_files(wqr_path)